In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%%time
from scrape_utils import WebScrapingUtilities
%run ../load_magic/storage.py
from html_analysis import HeaderAnalysis
from cypher_utils import CypherUtilities

wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']
s = Storage()
ha = HeaderAnalysis()
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

CPU times: total: 1.95 s
Wall time: 2.72 s


In [4]:

from datetime import datetime
from urllib.parse import urlparse, parse_qs
%run ../load_magic/dataframes.py
from html_analysis import ElementAnalysis
from IPython.display import clear_output

ea = ElementAnalysis()


----

In [5]:

file_path = '../data/html/indeed_email.html'
page_soup = get_page_soup(file_path)
css_selector = 'body > table > tbody > tr > td > a > table > tbody > tr > td > a'
link_soups_list = page_soup.select(css_selector)
files_list = []
for link_soup in link_soups_list:
    url_str = link_soup['href']
    page_soup = get_page_soup(url_str)
    page_title = page_soup.find('title').string.strip()
    file_name = re.sub(r'\W+', ' ', page_title).strip().replace(' ', '_')
    jk = parse_qs(urlparse(url_str).query).get('jk', [''])[0]
    if len(jk):
        file_name = f'{jk}_{file_name}.html'
    else:
        # file_name = datetime.now().strftime('%Y%m%d%H%M%S%f') + f'_{file_name}.html'
        file_name = f'{file_name}.html'
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    if not os.path.isfile(file_path):
        with open(file_path, 'w', encoding=s.encoding_type) as f:
            if verbose:
                print(f'Saving to {file_path}')
            f.write('<html><head><title>')
            f.write(page_title)
            f.write('</title></head><body>')
            row_div_list = page_soup.find_all(name='div', attrs={'class': ['jobsearch-JobComponent-description']})
            for div_soup in row_div_list:
                f.write(str(div_soup))
            f.write('</body></html>')
    files_list.append(file_name)

In [ ]:

verbose = True
cu.ensure_navigableparent('END', verbose=verbose)
for file_name in files_list:
    cu.ensure_filename(file_name, verbose=verbose)
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    for div_soup in row_div_list:
        child_strs_list = ha.get_navigable_children(div_soup, [])
        for sequence_order, (navigable_parent1, navigable_parent2) in enumerate(zip(child_strs_list[:-1], child_strs_list[1:])):
            clear_output(wait=True)
            cu.ensure_navigableparent(navigable_parent1, verbose=verbose)
            cu.ensure_navigableparents_relationship(navigable_parent1, navigable_parent2, file_name, sequence_order, verbose=verbose)
        
        # Add a fake relationship at the end
        clear_output(wait=True)
        cu.ensure_navigableparent(navigable_parent2, verbose=verbose)
        cu.ensure_navigableparents_relationship(navigable_parent2, 'END', file_name, sequence_order+1, verbose=verbose)
        
        child_tags_list = ha.get_child_tags_list(child_strs_list)
        for sequence_order, (header_tag1, header_tag2) in enumerate(zip(child_tags_list[:-1], child_tags_list[1:])):
            clear_output(wait=True)
            cu.ensure_headertag(header_tag1, verbose=verbose)
            cu.ensure_headertags_relationship(header_tag1, header_tag2, file_name, sequence_order, verbose=verbose)
        
        # Add a fake relationship at the end
        clear_output(wait=True)
        cu.ensure_headertag(header_tag2, verbose=verbose)
        cu.ensure_headertags_relationship(header_tag2, 'END', file_name, sequence_order+1, verbose=verbose)
        
        for sequence_order, (navigable_parent, header_tag) in enumerate(zip(child_strs_list, child_tags_list)):
            clear_output(wait=True)
            cu.ensure_headertag_navigableparent_relationship(header_tag, navigable_parent, verbose=verbose)